In [23]:
from experiment import ResultsViewer
from IPython.display import Image
import torch
from scipy.io import savemat, loadmat
import os
import json

In [24]:
results = ResultsViewer("experiments/server_e1")

In [25]:
def to_matlab(result):
    out = {}
    key = 'blocks.{}.{}.weight'
    sd = torch.load(result['state_dict_path'])
    out = {}
    for i in range(len(sd)//6):
        for sk in ['W', 'V', 'skip_l']:
            out[f'{sk}_{i}'] = sd[key.format(i, sk)].numpy()
    return out

def rewrite_matlab(all_results):
    rdir = all_results.results_dir
    for res in all_results:
        newmat = to_matlab(res)
        oldpath = res['matlab_path']
        fixpath = oldpath + '-fixed'
        savemat(fixpath, newmat)


In [30]:
for r in results:
    print(r['model']['linear_width'], r['model']['layers'], r['model']['regularization_lambda'])
    print(r['report']['Eval. MSE'])
    print("======")

1296 1 0.001
0.0
1296 1 0.1
0.33
1296 1 0.01
0.23
1296 1 0.001
0.0
1296 1 0.01
0.23
1296 1 0.1
0.33
1296 1 0.01
0.23
1296 1 0.1
0.33
1296 1 0.001
0.0
1296 1 0.001
0.0
1296 1 0.01
0.23
1296 1 0.1
0.33
16 2 0.001
0.0
16 2 0.1
0.33
16 2 0.01
0.0
16 1 0.001
0.0
16 1 0.01
0.23
16 1 0.1
0.33
16 2 0.001
0.0
16 2 0.01
0.0
16 2 0.1
0.33
1296 2 0.001
0.0
1296 2 0.01
0.0
1296 2 0.1
0.01
1296 2 0.001
0.0
1296 2 0.01
0.01
1296 2 0.1
0.02
16 1 0.001
0.0
16 1 0.01
0.23
16 1 0.1
0.33
16 2 0.001
0.0
16 2 0.01
0.0
16 2 0.1
0.33
